In [1]:
import os
import numpy as np
import jams
from scipy.io import wavfile
import sys
import librosa
from tensorflow.keras.utils import to_categorical

2022-10-12 22:33:20.534662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-12 22:33:20.534685: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path = "GuitarSet/"
path_audio = path + "audio/audio_mic/"
path_anno = path + "annotation/"

# labeling parameters
string_midi_pitches = [40,45,50,55,59,64]
highest_fret = 19
num_classes = highest_fret + 2 # for open/closed

# prepresentation and its labels storage
output = {}

# preprocessing modes
#
# c = cqt
# m = melspec
# cm = cqt + melspec
# s = stft
#
downsample = True
normalize = True
sr_downs = 22050

# CQT parameters
cqt_n_bins = 192
cqt_bins_per_octave = 24
hop_length = 512
preproc_mode = 'c'

In [3]:
def preprocess_audio(data):
    data = data.astype(float)
    if normalize:
        data = librosa.util.normalize(data)
    if downsample:
        data = librosa.resample(data, 44100, sr_downs)
        sr_curr = sr_downs
    if preproc_mode == "c":
        data = np.abs(librosa.cqt(data,
            hop_length=hop_length, 
            sr=sr_curr, 
            n_bins=cqt_n_bins, 
            bins_per_octave=cqt_bins_per_octave))
        print("invalid representation mode.")
    return data

In [4]:
def correct_numbering(n):
    n += 1
    if n < 0 or n > highest_fret:
        n = 0
    return n
def categorical(label):
    return to_categorical(label, num_classes)

In [5]:
def clean_label(label):
    label = [correct_numbering(n) for n in label]
    return categorical(label)

def clean_labels(labels):
    return np.array([clean_label(label) for label in labels])

In [32]:
def load_rep_and_labels_from_raw_file(filename):
        file_audio = path_audio + filename + "_mic.wav"
        file_anno = path_anno + filename + ".jams"
        jam = jams.load(file_anno)
        sr_original, data = wavfile.read(file_audio)
        sr_curr = sr_original
        
        # preprocess audio, store in output dict
        output["repr"] = np.swapaxes(preprocess_audio(data),0,1)
        
        # construct labels
        frame_indices = range(len(output["repr"]))
        times = librosa.frames_to_time(frame_indices, sr = sr_curr, hop_length=hop_length)
        
        # loop over all strings and sample annotations
        labels = []
        for string_num in range(1):
            anno = jam.annotations["note_midi"][5]
            string_label_samples = anno.to_samples(times)
            # replace midi pitch values with fret numbers
            for i in frame_indices:
                if string_label_samples[i] == []:
                    string_label_samples[i] = -1
                else:
                    string_label_samples[i] = int(round(string_label_samples[i][0]) - string_midi_pitches[5])
            labels.append([string_label_samples])
        print(labels)
        labels = np.array(labels)
        # remove the extra dimension 
        labels = np.squeeze(labels)
        labels = np.swapaxes(labels,0,1)
        
        # clean labels
        labels = clean_labels(labels)
    
        # store and return
        output["labels"] = labels
        return labels

In [33]:
temp = load_rep_and_labels_from_raw_file('00_BN1-129-Eb_comp')

invalid representation mode.
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

AxisError: axis2: axis 1 is out of bounds for array of dimension 1